----
# Apprentissage par renforcement - RLLIB et SAC

----
### Chargement des bibliothèques

In [ ]:
from ray.rllib.algorithms.sac import SACConfig
from ray.rllib.utils.replay_buffers.replay_buffer import ReplayBuffer
import matplotlib.pyplot as plt
from ray import tune
from IPython.display import display,clear_output
%matplotlib inline

----
### Configuration de l'entrainement

Les informations sont disponibles ici :

https://docs.ray.io/en/latest/rllib/rllib-training.html#rllib-config-rollouts

Chapitre __training__

Configurez :<br>
L'environnement Cartpole-v1<br>
Une machine d'entrainement (num_env_runner)<br>
Les hyperparamètres __gamma__ et __lr__<br>
Deux couches cachées avec 100 et 50 neurones pas exemples.<br>

Nous utiliserons **torch** et pas tensorflow.


In [ ]:
config = SACConfig()
config.environment("CartPole-v1")
config.env_runners(num_env_runners=1)
# config.framework("tf2")
config.training(gamma=0.98, lr=5e-02, train_batch_size=32,model={"fcnet_hiddens": [16 , 12]})
# Option avec grid search sur lr
# config = config.training(gamma=0.98, lr=tune.grid_search([1e-03,1e-04]), train_batch_size=100,model={"fcnet_hiddens": [100, 50]})
config.resources(num_gpus=1)

In [ ]:
config.model_config.values()

---
### Compiler la configuration

In [ ]:
algo=config.build()

---
### Lancez l'entrainement

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
plt.ion()
fig.show()
fig.canvas.draw()
rewards =[]
rewardsMax = []
for i in range(130):
    result = algo.train()
    # print(result['env_runners']['episode_reward_mean'])
    rewards.append(result['env_runners']['episode_reward_mean']) # 
    rewardsMax.append(max(result['env_runners']['hist_stats']['episode_reward']))
    # rewards_mean = [ sum(rewards[x:x+10])/10 for x in range(len(rewards)-10)]
    
    ax.clear()
    ax.plot(rewards)
    ax.plot(rewardsMax)

    fig.canvas.draw()
    clear_output(wait=True)
    display(fig)


In [ ]:
result

---
### Sauvegardez votre modèle 
(idéalement cela est fait fréquement pendant l'entrainment)

In [ ]:
algo.save(".")